In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.api as sm
import requests
import os.path
import time
import random
import re
import math
import sklearn


from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

In [2]:
#DOWLOADING FILE FROM DROPBOX FIRST TIME

while not os.path.exists('EX2_DATA_BASE.xlsx'):
    time.sleep (3*random.random()); #Sleeping less than 3 seconds before going to Dropbox - avoid too many students at once.
    if not os.path.exists('EX2_DATA_BASE.xlsx'):
        print "DOWLOADING FILE EX2_DATA_BASE.xlsx FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"

        #        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/EX2_DEV_DATA.xlsx")
        resp = requests.get("https://dl.dropboxusercontent.com/u/28535341/EX2_DATA_BASE.xlsx")        
        output = open('EX2_DATA_BASE.xlsx','wb')
        output.write(resp.content)
        output.close()
#DOWLOADING FILE FROM DROPBOX FIRST TIME

DOWLOADING FILE EX2_DATA_BASE.xlsx FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!


In [3]:
print "LOADING DATASETS..."

df = pd.read_excel(open('EX2_DATA_BASE.xlsx','rb'), sheetname='DATA')
df_dictionary = pd.read_excel(open('EX2_DATA_BASE.xlsx','rb'), sheetname='DICTIONARY')

LOADING DATASETS...


In [4]:
print "GETTING LIST OF VARIABLES..."

target_variable = str(df_dictionary[df_dictionary['TYPE_OF_VARIABLE'] == 'TARGET']['FIELD_NAME'].tolist()[0])

list_of_inputs_for_model0 = df_dictionary[df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0']['FIELD_NAME'].tolist()

list_of_inputs_for_model1 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')]['FIELD_NAME'].tolist()

list_of_inputs_for_model2 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')]['FIELD_NAME'].tolist()

list_of_inputs_for_model3 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT3')]['FIELD_NAME'].tolist()

list_of_inputs_for_model4 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT3')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT4')]['FIELD_NAME'].tolist()

GETTING LIST OF VARIABLES...


In [5]:
list_continuous_PD=[col for col in list(df) if (col.startswith('BB_1YR_DEFAULT_PROB'))]

In [6]:
# Se incluyen solo las variables continuas

list_of_inputs_for_model0 = [ v for v in list_of_inputs_for_model0 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]

list_of_inputs_for_model1 = [ v for v in list_of_inputs_for_model1 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model2 = [ v for v in list_of_inputs_for_model2 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model3 = [ v for v in list_of_inputs_for_model3 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model4 = [ v for v in list_of_inputs_for_model4 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]

In [7]:
df.shape #N<p

(116, 315)

In [39]:
# En este punto si tuviéramos un gran número de observaciones, separaríamos entre train y test.

In [8]:
# Comprobar que la Target está informada y en el rango (0,1)
df = df[(df[target_variable].between(0,1))] 

In [9]:
# Variable target y variable ID
target_variable='BB_1YR_DEFAULT_PROB_CQ4_2015'
identifier = 'TICKER'

In [10]:
df.shape

(111, 315)

In [11]:
# Tratamiento de missing value. Todas las variables son continuas por lo que se utilizn las medias.
# En el TFM habría que proceder a identificar en tipo de variables como en la práctica 1 de Gadi.
# Se procede a imputar valores sin estudiar si existen patrones. 
for var in df.columns:
    if (var in list_of_inputs_for_model4):
        missing=df[var].mean()
        df.loc[df[var].isnull(), var]=missing

In [12]:
# Tratamiento de "super outliers" 
# Se van a detectar a nivel univariante.  Un análisis más elaborado llevaría a la aplicación de técnicas multivariante. 
# 1. ¿Cómo se identifican? Criterio muy poco restrictivo si limpiamos a nivel univariante para no quedarnos sin observaciones. Optamos por un criterio simple y limpieza univariante.
# 2. Se sustituyen por el valor medio

# Outlier detection in very small sets, alternativamente MAD (mejorar para el TFM):
for var in df.columns:
    if (var in list_of_inputs_for_model4):
        media = df[var].mean()
        stdev= df[var].std()
        df.loc[abs(df[var] -media) > 3*stdev, var] = media            

In [13]:
# Construcción de ratios multiples limitado a: 
# 1. División entre todas las partidas del modelo del mismo año
# 2. Tasas de crecimiento de los ratios

In [118]:
### Modelo 0####

In [13]:
# Eliminar la target y el ID
dfid  = df [identifier]
dftarget = df[target_variable]

df.drop([identifier, target_variable], axis=1, inplace=True)
list0ratios = [x for x in list_of_inputs_for_model0 if x not in list_continuous_PD]

# Restringir el dataframe a las columnas contenidas en la lista
df_modelo0 = df[list_of_inputs_for_model0]

In [14]:
lenls=len(df_modelo0.columns)

for i in range(0, lenls):
    v_1=df_modelo0.columns[i]
    for j in range(i+1, lenls):
        v_2=df_modelo0.columns[j]
        if(v_1 in list0ratios and v_2 in list0ratios):
            ratio='r_div'+v_1+v_2
            df_modelo0[ratio]=np.where(df_modelo0[v_2]==0,0,(df_modelo0[v_1]/df_modelo0[v_2]))
            list_of_inputs_for_model0.append(ratio)

C:\Users\ico795\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
# Reemplazamos todos los NaN por la mediana de la columna por ser ratios (las obseervacione soriginales se habían sustituido por la media)

df_modelo0= df_modelo0.replace([np.inf,-np.inf],0)  # Repasar si se sustituye por la media. 
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [15]:
# Escalar las variables para aplicación de PCA
for var in df_modelo0.columns:
    if(var in list_of_inputs_for_model0):
        media=df_modelo0[var].mean()
        stdev=df_modelo0[var].std()
        
        df_modelo0[var]=(df_modelo0[var]-media) /stdev

In [16]:
# Realizadas todas las transformaciones, restringimos el df a ratios y PD para aplicar PCA

list_of_inputs_for_model0 = [ v for v in list_of_inputs_for_model0 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [206]:
# Habría que hacer un split en train y test en este punto

In [17]:
df_modelo0.shape

(111, 9875)

In [18]:
pca=PCA(n_components=len(df_modelo0))

In [19]:
df_pca=pca.fit_transform(df_modelo0)

In [20]:
df_pca0=pd.DataFrame(data=df_pca[::])

In [21]:
df_pca0.shape

(111, 111)

In [22]:
####### Modelo 0#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca0, dftarget)
    result=rf.fit(df_pca0, dftarget)
    y_pred = result.predict(df_pca0)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [23]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.6997542997542997, pvalue=1.2936820967309228e-17)


In [24]:
### Modelo 1####

In [25]:
# Lista de variables para el cálculo de ratios
list1ratios = [x for x in list_of_inputs_for_model1 if x not in list_continuous_PD]

In [27]:
df_modelo1 = df[list_of_inputs_for_model1]

In [28]:
lenls=len(df_modelo1.columns)

for i in range(0, lenls):
    v_1=df_modelo1.columns[i]
    for j in range(i+1, lenls):
        v_2=df_modelo1.columns[j]
        if(v_1 in list1ratios and v_2 in list1ratios):
            ratio='r_div'+v_1+v_2
            df_modelo1[ratio]=np.where(df_modelo1[v_2]==0,0,(df_modelo1[v_1]/df_modelo1[v_2]))
            list_of_inputs_for_model1.append(ratio)

C:\Users\ico795\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
df_modelo1= df_modelo1.replace([np.inf,-np.inf],0)

In [30]:
for var in df_modelo1.columns:
    if(var in list_of_inputs_for_model1):
        media=df_modelo1[var].mean()
        stdev=df_modelo1[var].std()
        df_modelo1[var]=(df_modelo1[var]-media) /stdev

In [31]:
list_of_inputs_for_model1 = [ v for v in list_of_inputs_for_model1 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [32]:
df_modelo1.shape

(111, 13536)

In [35]:
pca=PCA(n_components=len(df_modelo1))

In [36]:
df_pca=pca.fit_transform(df_modelo1)

In [41]:
df_pca1=pd.DataFrame(data=df_pca[::])

In [42]:
df_pca1.shape

(111, 111)

In [43]:
####### Modelo 1#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca1, dftarget)
    result=rf.fit(df_pca1, dftarget)
    y_pred = result.predict(df_pca1)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [44]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.78529308529308517, pvalue=1.9434671003796363e-24)


In [45]:
### Modelo 2####

In [46]:
# Eliminar la target y el ID
list2ratios = [x for x in list_of_inputs_for_model2 if x not in list_continuous_PD]

# Restringir el dataframe a las columnas contenidas en la lista
df_modelo2 = df[list_of_inputs_for_model2]

In [47]:
lenls=len(df_modelo2.columns)

for i in range(0, lenls):
    v_1=df_modelo2.columns[i]
    for j in range(i+1, lenls):
        v_2=df_modelo2.columns[j]
        if(v_1 in list2ratios and v_2 in list2ratios):
            ratio='r_div'+v_1+v_2
            df_modelo2[ratio]=np.where(df_modelo2[v_2]==0,0,(df_modelo2[v_1]/df_modelo2[v_2]))
            list_of_inputs_for_model2.append(ratio)
            

C:\Users\ico795\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
# Reemplazamos todos los NaN por la mediana de la columna por ser ratios (las obseervacione soriginales se habían sustituido por la media)

df_modelo2= df_modelo2.replace([np.inf,-np.inf],0)  # Repasar si se sustituye por la media. 
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [49]:
# Escalar las variables para aplicación de PCA
for var in df_modelo2.columns:
    if(var in list_of_inputs_for_model2):
        media=df_modelo2[var].mean()
        stdev=df_modelo2[var].std()
        
        df_modelo2[var]=(df_modelo2[var]-media) /stdev

In [50]:
# Realizadas todas las transformaciones, restringimos el df a ratios y PD para aplicar PCA

list_of_inputs_for_model2 = [ v for v in list_of_inputs_for_model2 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [51]:
df_modelo2.shape

(111, 17773)

In [52]:
pca=PCA(n_components=len(df_modelo2))

In [53]:
df_pca=pca.fit_transform(df_modelo2)

In [54]:
df_pca2=pd.DataFrame(data=df_pca[::])

In [55]:
df_pca2.shape

(111, 111)

In [56]:
####### Modelo 2#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca2, dftarget)
    result=rf.fit(df_pca2, dftarget)
    y_pred = result.predict(df_pca2)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [57]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.78886451386451384, pvalue=8.6485402026327006e-25)


In [118]:
### Modelo 3####

In [58]:
# Eliminar la target y el ID
list3ratios = [x for x in list_of_inputs_for_model3 if x not in list_continuous_PD]

# Restringir el dataframe a las columnas contenidas en la lista
df_modelo3 = df[list_of_inputs_for_model3]

In [59]:
lenls=len(df_modelo3.columns)

for i in range(0, lenls):
    v_1=df_modelo3.columns[i]
    for j in range(i+1, lenls):
        v_2=df_modelo3.columns[j]
        if(v_1 in list3ratios and v_2 in list3ratios):
            ratio='r_div'+v_1+v_2
            df_modelo3[ratio]=np.where(df_modelo3[v_2]==0,0,(df_modelo3[v_1]/df_modelo3[v_2]))
            list_of_inputs_for_model3.append(ratio)
            

C:\Users\ico795\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
# Reemplazamos todos los NaN por la mediana de la columna por ser ratios (las obseervacione soriginales se habían sustituido por la media)

df_modelo3= df_modelo3.replace([np.inf,-np.inf],0)  # Repasar si se sustituye por la media. 
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [61]:
# Escalar las variables para aplicación de PCA
for var in df_modelo3.columns:
    if(var in list_of_inputs_for_model3):
        media=df_modelo3[var].mean()
        stdev=df_modelo3[var].std()
        
        df_modelo3[var]=(df_modelo3[var]-media) /stdev

In [64]:
# Realizadas todas las transformaciones, restringimos el df a ratios y PD para aplicar PCA

list_of_inputs_for_model3 = [ v for v in list_of_inputs_for_model3 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [20]:
df_modelo3.shape

(111, 19605)

In [65]:
pca=PCA(n_components=len(df_modelo3))

In [66]:
df_pca=pca.fit_transform(df_modelo3)

In [67]:
df_pca3=pd.DataFrame(data=df_pca[::])

In [68]:
df_pca3.shape

(111, 111)

In [69]:
####### Modelo 3#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca3, dftarget)
    result=rf.fit(df_pca3, dftarget)
    y_pred = result.predict(df_pca3)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [71]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.75873113373113366, pvalue=5.1167524404165563e-22)


In [118]:
### Modelo 4####

In [73]:
# Eliminar la target y el ID
list4ratios = [x for x in list_of_inputs_for_model4 if x not in list_continuous_PD]

# Restringir el dataframe a las columnas contenidas en la lista
df_modelo4 = df[list_of_inputs_for_model4]

In [74]:
lenls=len(df_modelo4.columns)

for i in range(0, lenls):
    v_1=df_modelo4.columns[i]
    for j in range(i+1, lenls):
        v_2=df_modelo4.columns[j]
        if(v_1 in list4ratios and v_2 in list4ratios):
            ratio='r_div'+v_1+v_2
            df_modelo4[ratio]=np.where(df_modelo4[v_2]==0,0,(df_modelo4[v_1]/df_modelo4[v_2]))
            list_of_inputs_for_model4.append(ratio)
            

C:\Users\ico795\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
# Reemplazamos todos los NaN por la mediana de la columna por ser ratios (las obseervacione soriginales se habían sustituido por la media)

df_modelo4= df_modelo4.replace([np.inf,-np.inf],0)  # Repasar si se sustituye por la media. 
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [76]:
# Escalar las variables para aplicación de PCA
for var in df_modelo4.columns:
    if(var in list_of_inputs_for_model4):
        media=df_modelo4[var].mean()
        stdev=df_modelo4[var].std()
        
        df_modelo4[var]=(df_modelo4[var]-media) /stdev

In [77]:
# Realizadas todas las transformaciones, restringimos el df a ratios y PD para aplicar PCA
list_of_inputs_for_model4 = [ v for v in list_of_inputs_for_model4 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [78]:
df_modelo4.shape

(111, 30389)

In [79]:
pca=PCA(n_components=len(df_modelo4))

In [80]:
df_pca=pca.fit_transform(df_modelo4)

In [81]:
df_pca4=pd.DataFrame(data=df_pca[::])

In [82]:
df_pca4.shape

(111, 111)

In [83]:
####### Modelo 4#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca4, dftarget)
    result=rf.fit(df_pca4, dftarget)
    y_pred = result.predict(df_pca4)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [85]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.82120919620919619, pvalue=2.5605345845928001e-28)
